# Logistic Regression

In [1]:
import torch
import torch.nn as nn
import numpy as np

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [9]:
#0) Prepare data
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape
#print(n_samples, n_features) #569 samples with 30 features

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2022)

#Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#Convert to Torch
X_train = torch.from_numpy(X_train.astype(np.float32)) #need to convert to float 
y_train = torch.from_numpy(y_train.astype(np.float32)) #need to convert to float
X_test = torch.from_numpy(X_test.astype(np.float32))   #need to convert to float
y_test = torch.from_numpy(y_test.astype(np.float32))   #need to convert to float

#Convert y into from [0,1,2,3] ==> [[0], [1], [1]]
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

#1) Model: f = wx + b, sigmoid at the end
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features, output_size):
        #super(Model, self).__init__() = super().__init__(self)
        #this allows to init the model without any input paras: model = LogisticRegression()
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, output_size)
    
    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted

input_size = n_features
output_size = 1

model = LogisticRegression(n_features, output_size)

#2) Loss and Optimizer
num_epochs = 100
learning_rate = 0.01
criterion = nn.BCELoss() #binary cross-entropy
optimmizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

#3) Training Loop

for epoch in range(num_epochs):
    #forward pass & loss
    y_predicted = model.forward(X_train)
    loss = criterion(y_predicted, y_train)
    #backward pass
    loss.backward()

    #weight update
    optimmizer.step()
    optimmizer.zero_grad()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1:>3,}: Loss: {loss.item():.3f}")

#Evaludation:
with torch.no_grad(): #to ensure this is not in the computation graph
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round() #Since sigmoid return btw 0~1
    acc = y_predicted_cls.eq(y_test).sum()/float(y_test.shape[0])
    print(f"Accuracy = {acc:.4f}")



Epoch  10: Loss: 0.569
Epoch  20: Loss: 0.478
Epoch  30: Loss: 0.418
Epoch  40: Loss: 0.376
Epoch  50: Loss: 0.344
Epoch  60: Loss: 0.320
Epoch  70: Loss: 0.300
Epoch  80: Loss: 0.284
Epoch  90: Loss: 0.270
Epoch 100: Loss: 0.258
Accuracy = 0.9737
